Compare two ways of estimating user- and item- bias factors
1. One hot encode them and put them into an xgboost model with some other features
2. Use `diamond` to fit a crossed random-effects model, then put these predictions into an xgboost model with the same set of other features

These other features include
* genre encoding: one song can be in multiple genres
* Categorical features
    * source_system_tab
    * source_screen_name
    * source_type 
    * artist_name
    * composer
    * lyricist
    * language
    * city
    * gender
    * registered_via
* Numeric features
    * song_length
    * bd
    * registration_init_time
    * expiration_date

In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter, defaultdict

from scipy import sparse
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score

from diamond.glms.logistic import LogisticRegression
import xgboost as xgb
import utils

import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s',
                   level=logging.INFO)

/Users/timothysweetser/python/virtualenv/work3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Using [nodebook](http://github.com/stitchfix/nodebook)

In [2]:
def id_to_index(ids):
    " return a dict like {id: idx} "
    unique_ids = set(ids)
    return dict(zip(unique_ids, np.arange(0, len(unique_ids))))

def encode_categoricals(df, cat_features, id_col):
    """
    one hot encode categorical features. modifies `df` in-place
    """
    idx = id_col + 'x'
    for x in cat_features:
        df[x] = df[x].astype(str).fillna('unknown')
    id_map = id_to_index(df[id_col])
    df[idx] = df[id_col].apply(lambda x: id_map[x])
    df = df[cat_features + [idx]].\
        sort_values(idx)
    X = DictVectorizer().fit_transform(df.T.to_dict().values())
    return X.tocsr(), id_map

def encode_genres(song_map):
    " this is a many-hot encoding of song to genre "
    df_genres = df_songs['genre_ids'].apply(lambda s: pd.Series(str(s).split('|')))
    df_genres['song_idx'] = df_songs['song_id'].apply(lambda x: song_map[x])
    df_genres2 = pd.melt(df_genres, 'song_idx', value_name='genre').\
        drop('variable', axis=1).\
        dropna().\
        sort_values('song_idx')
    genre_map = id_to_index(df_genres2['genre'])
    df_genres2['genre_idx'] = df_genres2['genre'].apply(lambda g: genre_map[g])
    X_genres = sparse.coo_matrix((np.ones(len(df_genres2)),
                                 (df_genres2['song_idx'], df_genres2['genre_idx'])))
    return X_genres.tocsr(), genre_map

# 1. Read in data

In [3]:
df_train = pd.read_csv('data/raw/train.csv')
df_songs = pd.read_csv('data/raw/songs.csv')
df_members = pd.read_csv('data/raw/members.csv')

# 2. Create design matrices

### Songs

In [4]:
%%time
X_songs, song_map = encode_categoricals(df_songs, 
                              ['artist_name', 'composer', 'lyricist', 'language'],
                              'song_id')
assert X_songs.shape[0] == len(df_songs)
print(X_songs.shape)

(2296320, 663127)
CPU times: user 3min 12s, sys: 3.21 s, total: 3min 15s
Wall time: 3min 15s


### Members

In [5]:
%%time
X_members, member_map = encode_categoricals(df_members,
                                ['city', 'gender', 'registered_via'],
                                'msno')
assert X_members.shape[0] == len(df_members)
print(X_members.shape)

(34403, 31)
CPU times: user 2.38 s, sys: 53.7 ms, total: 2.43 s
Wall time: 2.43 s


### IDs: benchmark for diamond

In [6]:
%%time
X_member_ids = sparse.coo_matrix((np.ones(len(df_members)),
                                  (df_members['msnox'], df_members['msnox']))).tocsr()
X_song_ids = sparse.coo_matrix((np.ones(len(df_songs)),
                                  (df_songs['song_idx'], df_songs['song_idx']))).tocsr()


CPU times: user 541 ms, sys: 60.4 ms, total: 601 ms
Wall time: 601 ms


### Genres

In [7]:
%%time
X_genres, genre_map = encode_genres(song_map)

CPU times: user 7min 5s, sys: 17.2 s, total: 7min 23s
Wall time: 7min 24s


### Source features
Thanks to [Ritchie Ng](http://www.ritchieng.com/machinelearning-one-hot-encoding/) for guidance on using One Hot Encoder

In [8]:
%%time
source_features = ['source_system_tab', 'source_screen_name', 'source_type']
LE, OHE = LabelEncoder(), OneHotEncoder()

X_source = OHE.fit_transform(
    df_train[source_features].\
    fillna('unknown').\
    apply(LE.fit_transform)
)

CPU times: user 39.9 s, sys: 2.2 s, total: 42.1 s
Wall time: 42.2 s


# 3. Train diamond model

Need a train/test split for this and subsequent modeling

In [9]:
df_val = df_train.iloc[utils.TTS:, :]
df_train = df_train.iloc[:utils.TTS, :]

In [10]:
%%time
formula = 'target ~ 1 + (1|song_id) + (1|msno)'

priors = pd.DataFrame({'group': ['song_id', 'msno'],
                       'var1': ['intercept'] * 2,
                       'var2': [np.nan] * 2,
                       # fit on a sample of data in R/lme4
                       'vcov': [0.00845, 0.07268]})
diamond = LogisticRegression(df_train, priors)
effects = diamond.fit(formula, tol=1e-5, verbose=False, max_its=200)

2017-10-22 14:46:10,034 INFO:creating main design matrix
2017-10-22 14:46:12,057 INFO:creating song_id design matrix
2017-10-22 14:46:23,091 INFO:creating msno design matrix
2017-10-22 14:46:30,198 INFO:creating covariance matrix
2017-10-22 14:46:30,202 INFO:creating Hessians
2017-10-22 14:46:30,297 INFO:creating H_inter for song_id
2017-10-22 14:46:31,637 INFO:time elapsed: 53.8
2017-10-22 14:46:31,638 INFO:blocks inverted: 0 of 287748
2017-10-22 14:46:57,719 INFO:time elapsed: 79.9
2017-10-22 14:46:57,720 INFO:blocks inverted: 100000 of 287748
2017-10-22 14:47:23,817 INFO:time elapsed: 106.0
2017-10-22 14:47:23,817 INFO:blocks inverted: 200000 of 287748
2017-10-22 14:47:46,768 INFO:creating H_invs
2017-10-22 14:47:47,614 INFO:creating H_inter for msno
2017-10-22 14:47:48,173 INFO:time elapsed: 130.4
2017-10-22 14:47:48,174 INFO:blocks inverted: 0 of 26411
2017-10-22 14:47:55,106 INFO:creating H_invs
2017-10-22 14:54:02,598 INFO:extracting coefficients


CPU times: user 7min 26s, sys: 1min 45s, total: 9min 12s
Wall time: 8min 25s


In [11]:
with open('models/diamond.p', 'wb') as ff:
    pickle.dump(diamond, ff)

df_train['diamond_pred'] = diamond.predict(df_train)
df_val['diamond_pred'] = diamond.predict(df_val)

In [12]:
df_train.drop(['row_index', 'intercept'], axis=1, inplace=True)

# 4. Put everything together

In [ ]:
def merge_it_all_together(df, use_diamond):
    """
    Merge member, song, and source data together
    Return (X, y) tuple
    This makes a _copy_ of df
    """
    
    logging.info('merging in song info')
    df = pd.merge(df.drop(source_features, axis=1),
                  df_songs[['song_id', 'song_idx', 'song_length']],
                  # there are 80 interactions with unknown songs in the training set
                  # remove them!
                  'inner',
                  'song_id')
    logging.info('merging in member info')
    df = pd.merge(df, df_members[['msno', 'msnox',
                                  'registration_init_time', 'expiration_date', 'bd']], 'left', 'msno')
    n_nulls = df.isnull().sum()
    assert n_nulls.sum() == 0, n_nulls
    logging.info('there are no nulls')
    
    numeric_features = ['song_length', 'registration_init_time', 'expiration_date', 'bd']
    X_numeric = df[numeric_features].as_matrix()
    member_idx = df['msnox']
    song_idx = df['song_idx']
    del df  # free up the memory
    
    if use_diamond:
        logging.info('using diamond predictions')
        numeric_features.append('diamond_prediction')
        matrix_list = []
    else:
        logging.info('using one-hot encoding of member, song ids')
        matrix_list = [X_member_ids[member_idx, :],
                       X_song_ids[song_idx, :]]
    
    logging.info('adding numeric feature matrix')
    matrix_list.append(X_numeric)
    logging.info('creating song matrix')
    matrix_list.append(X_songs[song_idx, :])
    logging.info('creating genre matrix')
    matrix_list.append(X_genres[sonx_idx, :])
    logging.info('creating member matrix')
    matrix_list.append(X_members[member_idx, :])

    logging.info('concatenating matrices')
    X = sparse.hstack(matrix_list)
    
    y = df['target']
    return X, y

In [ ]:
X_val, y_val = merge_it_all_together(df_val, use_diamond=True)
del df_val
D_val = xgb.DMatrix(sparse.hstack([X_val, X_source[utils.TTS:, :]]))

2017-10-22 14:54:35,851 INFO:merging in song info
2017-10-22 14:54:38,249 INFO:merging in member info
2017-10-22 14:54:40,137 INFO:there are no nulls
2017-10-22 14:54:40,159 INFO:using diamond predictions
2017-10-22 14:54:40,160 INFO:adding numeric feature matrix
2017-10-22 14:54:40,161 INFO:creating song matrix


In [ ]:
%%time
X_train, y_train = merge_it_all_together(df_train, use_diamond=False)
D_train = xgb.DMatrix(sparse.hstack([X_train, X_source[:utils.TTS, :]]),
                      y_train)
del X_train, y_train

### 4. Train xgboost models

# TODO
Use 5-fold CV and grid search to estimate hyperparameters

In [ ]:
# df_train model, evaluate and make predictions
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.75
params['max_depth'] = 5
params['silent'] = 1
params['eval_metric'] = 'auc'
MAX_ITS = 200

model_without_diamond = xgb.train(params, D_train, MAX_ITS)

Fit a similar model, but using diamond's predictions instead of one-hot-encoded song and member id's

In [ ]:
X_train, y_train = merge_it_all_together(df_train, use_diamond=True)
X_train = sparse.hstack([X_train, X_source[:utils.TTS, :]])
D_train = xgb.DMatrix(X_train, y_train)
del X_train, y_train
X_val, y_val = merge_it_all_together(df_val, use_diamond=True)
X_val = sparse.hstack([X_val, X_val[utils.TTS:, :]])
D_val = xgb.DMatrix(X_val, y_val)

In [ ]:
model_with_diamond = xgb.train(params, D_train, MAX_ITS)

## Compare predictions

In [ ]:
roc_auc(score(df_val['target']),
        model_without_diamond.predict(D_val))

In [ ]:
roc_auc(score(df_val['target']),
        model_with_diamond.predict(D_val))